### Jupyter notebook for capturing a discussion (or possibily own thoughts)

In [1]:
import pyaudio
import sounddevice as sd
import numpy as np
import speech_recognition as sr
import whisper
# import torch

from typing import List, Dict, Any

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import PyPDFLoader


import LIB_AudioLib as myALib

import asyncio
import nest_asyncio

# Apply nest_asyncio if needed
nest_asyncio.apply()
import nest_asyncio

from datetime import datetime
import shutil

In [2]:
# GPU-Speicher freigeben
# torch.cuda.empty_cache()

# Load the Whisper model
model = whisper.load_model("medium", device="cpu")

In [3]:
output_devices, default_device_index = myALib.get_audio_output_devices()

for device in output_devices:
    print(f"Device Index: {device['index']}, Device Name: {device['name']}")

# Format the options for the radio button
options = [f"{device['name']} (Index: {device['index']})" for device in output_devices]

# Determine the default index for the radio button
if default_device_index is not None:
    default_index = next((i for i, d in enumerate(output_devices) if d['index'] == default_device_index), 0)
else:
    default_index = 0

Device Index: 0, Device Name: HDA Intel PCH: ALC1150 Analog (hw:0,0)
Device Index: 1, Device Name: HDA Intel PCH: ALC1150 Digital (hw:0,1)
Device Index: 3, Device Name: Plantronics Blackwire 3220 Seri: USB Audio (hw:1,0)
Device Index: 4, Device Name: HDA NVidia: HDMI 0 (hw:2,3)
Device Index: 5, Device Name: HDA NVidia: HDMI 1 (hw:2,7)
Device Index: 6, Device Name: HDA NVidia: HDMI 2 (hw:2,8)
Device Index: 7, Device Name: HDA NVidia: HDMI 3 (hw:2,9)
Device Index: 8, Device Name: Jabra SPEAK 510 USB: Audio (hw:3,0)
Device Index: 9, Device Name: Dell Universal Dock D6000: USB Audio (hw:4,0)
Device Index: 11, Device Name: sysdefault
Device Index: 12, Device Name: front
Device Index: 13, Device Name: surround21
Device Index: 14, Device Name: surround40
Device Index: 15, Device Name: surround41
Device Index: 16, Device Name: surround50
Device Index: 17, Device Name: surround51
Device Index: 18, Device Name: surround71
Device Index: 19, Device Name: iec958
Device Index: 20, Device Name: spdif

In [4]:
# Example usage: Generate a sound on device for 1 second
selected_device_index = 8  # Replace with the index of the device you want to use

In [5]:
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
text_filename=f"text_{timestamp}.txt"
speech_length = 1

while speech_length > 0.5:
    # Generate a sound on the selected device for 1 second
    myALib.generate_sound_on_device(device_index=selected_device_index, duration=1)

    # Capture question from the microphone for 10 seconds and save it to "input.wav"
    now = datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    speech_length = myALib.capture_speech_from_microphone(duration=0, output_filename=f"input_{timestamp}.wav")
    print(f"Speech length: {speech_length:.2f} seconds")

    if speech_length > 0.5:
        result = model.transcribe(f"input_{timestamp}.wav")

        # Save the transcription to a text file
        with open(text_filename, "a") as text_file:
            text_file.write(f"{result["text"]}\n\n")
        # Print the transcription
        print(result['text'])

Recording...
Silence detected.
Speech length: 88.32 seconds


/home/gabriel/myProject_LLM/myvenv_llm/lib/python3.12/site-packages/whisper/transcribe.py:130: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/gabriel/myProject_LLM/myvenv_llm/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Jetzt möchte ich die Kamerastrategie beschreiben. Also wir haben drei unterschiedliche Software Module. Die eine ist die App, das zweite ist die Firmware in der Kamera, die dritte ist die Cloud Software. In der Firmware für die Kamera haben wir zwei Teile, das Betriebssystem und die Home spezifische Lösung. In der Cloud Software haben wir zwei Module, eine ist den Residential Cloud, der behandelt alle Funktionen die wir in dem spezifisch für die Home Kamera haben und den BT Cloud, das ist der Video Relay und der Video Relay beinhaltet eigentlich die Aufnahme der Clips, die Speicherung der Clips in einem Speicher und die Indexierung mit weiterer Information wie Daten und so weiter.
Recording...
Silence detected.
Speech length: 0.21 seconds


In [ ]:
del model  # Modell löschen
# torch.cuda.empty_cache()  # Speicher freigeben

In [ ]:
import pyaudio

def check_supported_sample_rates(device_index):
    p = pyaudio.PyAudio()
    supported_rates = []
    for rate in [8000, 16000, 22050, 32000, 44100, 48000, 96000]:
        try:
            if p.is_format_supported(rate,
                                     output_device=device_index,
                                     output_channels=1,
                                     output_format=pyaudio.paInt16):
                supported_rates.append(rate)
        except ValueError:
            pass
    p.terminate()
    return supported_rates

# Check supported sample rates for device 8
device_index = 8
supported_rates = check_supported_sample_rates(device_index)
print(f"Supported sample rates for device {device_index}: {supported_rates}")

Supported sample rates for device 8: [8000, 16000, 48000]


In [8]:
import pyaudio

def check_supported_sample_rates(device_index):
    p = pyaudio.PyAudio()
    supported_rates = []
    for rate in [8000, 16000, 22050, 32000, 44100, 48000, 96000]:
        try:
            if p.is_format_supported(rate,
                                     output_device=device_index,
                                     output_channels=1,
                                     output_format=pyaudio.paInt16):
                supported_rates.append(rate)
        except ValueError:
            pass
    p.terminate()
    return supported_rates

# Check supported sample rates for device 8
device_index = 3
supported_rates = check_supported_sample_rates(device_index)
print(f"Supported sample rates for device {device_index}: {supported_rates}")

Supported sample rates for device 3: [8000, 16000, 32000, 44100, 48000]


#Prompt:

Korrigier und Formatier bitte folgenden Speech-to-text Text in klaren Abschnitten mit Überschriften, Numerierung, Auflistung und Unterpunkten